# StochSS Analysis for Stochastic Parameter Sweep

In [ ]:
%matplotlib
%matplotlib inline
import numpy
import sys
from matplotlib import pyplot as plt
import pickle
import pyurdme

In [ ]:
import stochss_parametersweep_program

In [ ]:
parameters = stochss_parametersweep_program.getParameters()

In [ ]:
sweep_data = pickle.load(open('results'))
species_names = sweep_data[0]['result']['max'].keys()
print 'Species:',species_names
parameter_names = sweep_data[0]['parameters'].keys()
print 'Parameters:',parameter_names

In [ ]:
species_ndx = 0
mapper_key = 'avg'
reducer_key = 'avg'
if len(parameter_names) == 1:
    plotdata = numpy.zeros(len(sweep_data))
    xdata = numpy.zeros(len(sweep_data))
    for i,d in enumerate(sweep_data):
        plotdata[i] = d['result'][mapper_key][species_names[0]][reducer_key]
        xdata[i] = d['parameters'][parameter_names[0]]
    fig, ax = plt.subplots(figsize=(8,8))
    plt.title("Species: {0} Analysis {1}-{2}".format(species_names[species_ndx], mapper_key, reducer_key))
    plt.plot(xdata,plotdata)
    plt.xlabel(parameter_names[0], fontsize=16, fontweight='bold')
else:
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    heatmap = numpy.zeros((len(parameters[parameter_names[0]]),len(parameters[parameter_names[1]])))
    for p0ndx,p0val in enumerate(parameters[parameter_names[0]]):
        for p1ndx,p1val in enumerate(parameters[parameter_names[1]]):
            heatmap[p0ndx,p1ndx] = numpy.nan
            for i in range(len(sweep_data)):
                if sweep_data[i]['parameters'] == {parameter_names[0]:p0val, parameter_names[1]:p1val}:
                    heatmap[p0ndx,p1ndx] = sweep_data[i]['result'][mapper_key][species_names[species_ndx]][reducer_key]
                    break
    fig, ax = plt.subplots(figsize=(8,8))
    plt.imshow(heatmap)
    ax.set_xticks(numpy.arange(heatmap.shape[1])+0.5, minor=False)
    ax.set_yticks(numpy.arange(heatmap.shape[0])+0.5, minor=False)
    plt.title("Species: {0} Analysis {1}-{2}".format(species_names[species_ndx], mapper_key, reducer_key))
    ax.set_xticklabels(parameters[parameter_names[1]], minor=False, rotation=90)
    ax.set_yticklabels(parameters[parameter_names[0]], minor=False)
    ax.set_xlabel(parameter_names[1], fontsize=16, fontweight='bold')
    ax.set_ylabel(parameter_names[0], fontsize=16, fontweight='bold')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.2)
    _ = plt.colorbar(ax=ax, cax=cax)

# Custom Post-processing example

In [ ]:
def exampleMapperFunction(result):
    # return the value at the final timepoint for each species
    mapped_data = numpy.zeros(len(result.keys()))
    for i,s in enumerate(sorted(result.keys())):
        mapped_data[i] = result[s][-1]
    return mapped_data

In [ ]:
def exampleReducerFunction(mapped_data_list):
    # Calculate the coefficient of variation for each species
    num_trajectories = len(mapped_data_list)
    num_species = len(mapped_data_list[0])
    mapped_data_list = numpy.array(mapped_data_list)
    reduced_data = numpy.zeros(num_species)
    if num_trajectories == 1:
        return numpy.array([0]*num_species)
    else:
        for i in range(num_species):
            reduced_data[i] = numpy.sqrt(numpy.var(mapped_data_list[:,i]))/numpy.mean(mapped_data_list[:,i])
    return reduced_data

In [ ]:
%time example_sweep_data = stochss_parametersweep_program.run_local_parameter_sweep(parameters, mapper_fn=exampleMapperFunction, reducer_fn=exampleReducerFunction )

In [ ]:
species_ndx = 0
if len(parameter_names) == 1:
    plotdata = numpy.zeros(len(example_sweep_data))
    xdata = numpy.zeros(len(example_sweep_data))
    for i,d in enumerate(example_sweep_data):
        plotdata[i] = d['result'][species_ndx]
        xdata[i] = d['parameters'][parameter_names[0]]
    fig, ax = plt.subplots(figsize=(8,8))
    plt.title("Species: {0} Analysis {1}-{2}".format(species_names[species_ndx], mapper_key, reducer_key))
    plt.plot(xdata,plotdata)
    plt.xlabel(parameter_names[0], fontsize=16, fontweight='bold')
else:
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    heatmap = numpy.zeros((len(parameters[parameter_names[0]]),len(parameters[parameter_names[1]])))
    for p0ndx,p0val in enumerate(parameters[parameter_names[0]]):
        for p1ndx,p1val in enumerate(parameters[parameter_names[1]]):
            heatmap[p0ndx,p1ndx] = numpy.nan
            for i in range(len(example_sweep_data)):
                if example_sweep_data[i]['parameters'] == {parameter_names[0]:p0val, parameter_names[1]:p1val}:
                    heatmap[p0ndx,p1ndx] = example_sweep_data[i]['result'][species_ndx]
                    break
    fig, ax = plt.subplots(figsize=(8,8))
    plt.imshow(heatmap)
    ax.set_xticks(numpy.arange(heatmap.shape[1])+0.5, minor=False)
    ax.set_yticks(numpy.arange(heatmap.shape[0])+0.5, minor=False)
    plt.title("Analyzing species: {0}".format(species_names[species_ndx]))
    ax.set_xticklabels(parameters[parameter_names[1]], minor=False, rotation=90)
    ax.set_yticklabels(parameters[parameter_names[0]], minor=False)
    ax.set_xlabel(parameter_names[1], fontsize=16, fontweight='bold')
    ax.set_ylabel(parameter_names[0], fontsize=16, fontweight='bold')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.2)
    _ = plt.colorbar(ax=ax, cax=cax)